In [ ]:
pip install azure-storage-file-datalake pyarrow git+https://github.com/CDCgov/phdi

In [ ]:
from azure.identity import ManagedIdentityCredential
from azure.storage.filedatalake import DataLakeFileClient
from azure.core.credentials import AccessToken
import time
import sys
from phdi.linkage.seed import convert_to_patient_fhir_resources
import pyarrow.parquet as pq
import pyarrow as pa

# Set up authentication
class spoof_token:
    def get_token(*args, **kwargs):
        return AccessToken(
            token=mssparkutils.credentials.getToken(audience="storage"),
            expires_on=int(time.time())+60*10
        )

credential = ManagedIdentityCredential()
credential._credential = spoof_token()

# Set up file client
account_name = "phdidevphi9d194c64"
file_system_name = "source-data"
file_path = "synthetic_patient_mpi_seed_data.parquet"
file_client = DataLakeFileClient(account_url=f"https://{account_name}.dfs.core.windows.net",
                                 file_system_name=file_system_name,
                                 file_path=file_path,
                                 credential=credential)

# This function converts parquet bytes into FHIR bundles grouped by IRIS ID
def convert(bytes):
    buffer = pa.BufferReader(bytes)
    parquet_file = pq.ParquetFile(buffer)
    converted_data = {}
    for row in parquet_file.iter_batches(batch_size=1):
        data = row.to_pylist()[0]
        iris_id, fhir_bundle = convert_to_patient_fhir_resources(data)
        converted_data[iris_id] = fhir_bundle
    
    return converted_data

# Download parquet file and convert
download = file_client.download_file()
bytes = download.readall()

converted_data = convert(bytes)

print(converted_data)